# Subjectivity classification with CNNs

In this notebook we implement the approched described in this [paper](https://arxiv.org/pdf/1408.5882.pdf) for classifiying sentences using Convolutional Neural Networks. In particular, we will classify sentences into "subjective" or "objective". 

## Subjectivity Dataset

The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [1]:
from pathlib import Path
PATH = Path("/Users/yinterian/teaching/deeplearning/data/rotten_imdb/")
list(PATH.iterdir())

[PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.100d.txt'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.200d.txt'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.300d.txt'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.50d.txt'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.zip'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/plot.tok.gt9.5000'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/quote.tok.gt9.5000'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/rotten_imdb.tar'),
 PosixPath('/Users/yinterian/teaching/deeplearning/data/rotten_imdb/subjdata.README.1.0')]

From the readme file:
- quote.tok.gt9.5000 contains 5000 subjective sentences (or snippets)
- plot.tok.gt9.5000 contains 5000 objective sentences

In [2]:
! head /Users/yinterian/teaching/deeplearning/data/rotten_imdb/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 
spurning her mother's insistence that she get on with her life , mary is thrown out of the house , rejected by joe , and expelled from school as she grows larger with child . 
amitabh can't believe the board of directors and his mind is filled with revenge and what better revenge than robbing the bank himself , ironic as it may sound . 
she , among others excentricities , talks to a small rock , gertrude , like if she was alive . 
this gives the girls a fair chance of pulling the wool over their eyes using their sexiness to poach any last vestige of common sense the dons might have had . 
styled after vh1's " behin

## String cleaning functions

In [3]:
import numpy as np
from collections import defaultdict
import re

In [4]:
# this is from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
def clean_str(string):
    """Tokenization/string cleaning. """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [5]:
def read_file(path):
    """ Read file returns a shuttled list.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = np.array(f.readlines())
    return content

In [6]:
def get_vocab(list_of_content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for content in list_of_content:
        for line in content:
            line = clean_str(line.strip())
            words = set(line.split())
            for word in words:
                vocab[word] += 1
    return vocab       

## Split train and test

In [7]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([clean_str(line.strip()) for line in sub_content])
obj_content = np.array([clean_str(line.strip()) for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path \\?',
        "the director 's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship \\( most notably wretched sound design \\)",
        "welles groupie scholar peter bogdanovich took a long time to do it , but he 's finally provided his own broadside at publishing giant william randolph hearst",
        'based on the 1997 john king novel of the same name with a rather odd synopsis a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack'],
       dtype='<U679'), array([1., 0., 0., 1., 1.]))

In [107]:
# getting vocab from training sets
data_vocab = get_vocab([X_train])

In [101]:
#data_vocab

## Embedding Layer

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
# an Embedding module containing 10 (words) tensors of size 3
embed = nn.Embedding(10, 3)
a = torch.LongTensor([[1,2,4,5,1]])
embed(a)

tensor([[[-0.5333,  0.7776, -1.2749],
         [ 0.6352, -1.5186,  1.0346],
         [-0.2285,  0.1224, -1.9241],
         [ 1.2994, -1.6922,  1.8019],
         [-0.5333,  0.7776, -1.2749]]])

In [16]:
## here is the randomly initialized embeddings
embed.weight.data

tensor([[ 0.9265, -0.1933,  1.3762],
        [-0.5333,  0.7776, -1.2749],
        [ 0.6352, -1.5186,  1.0346],
        [-1.1091,  0.4668,  0.3990],
        [-0.2285,  0.1224, -1.9241],
        [ 1.2994, -1.6922,  1.8019],
        [-0.2287,  1.5780, -0.3959],
        [ 0.3668, -0.6783,  0.3945],
        [ 0.7854,  1.9058, -0.8052],
        [ 0.7321, -1.2108,  0.5928]])

### Initializing embedding layer with Glove embeddings

To get glove pre-trained embeddings:
    `wget http://nlp.stanford.edu/data/glove.6B.zip`

In this section I am keeping the whole Glove embeddings. You can decide to keep just the words on your training set.

In [18]:
! head -2 /Users/yinterian/teaching/deeplearning/data/rotten_imdb/glove.6B.50d.txt

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581
, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392


We would like to initialize the embeddings from our model with the pre-trained Glove embeddings. After initializing we should "freeze" the embeddings at least initially. The rationale is that we first want the network to learn weights for the other parameters that were randomly initialize. After that phase we could finetune the embeddings to our task. 

`embed.weight.requires_grad = False` freezes the embedding parameters.

The following code initializes the embedding. Here `V` is the vocabulary size and `D` is the embedding size. `pretrained_weight` is a numpy matrix of shape `(V, D)`.

In [19]:
def loadGloveModel(gloveFile=PATH/"glove.6B.300d.txt"):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [20]:
word_vecs = loadGloveModel()

In [21]:
print(len(word_vecs.keys()), len(data_vocab.keys()))

400000 19310


In [159]:
def delete_rare_words(word_vecs, data_vocab, min_df=2):
    """ Deletes rare words from data_vocab
    
    Deletes words from data_vocab if they are not in word_vecs
    and don't have at least min_df occurrencies in data_vocab.
    """
    words_delete = []
    for word in data_vocab:
        if data_vocab[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    print(words_delete)
    for word in words_delete: data_vocab.pop(word)
    return data_vocab

In [160]:
len(data_vocab.keys())

19310

In [161]:
# clean up issues here
data_vocab = delete_rare_words(word_vecs, data_vocab, min_df=2) 

["ragam'", "'mouna", 'maniratnam', "'alaipayuthe'", "'pound", "flesh'", 'glizty', "'top", 'zidanes', "meyjes'", 'haphazardness', 'sepideh', 'oddyssey', 'kervorkian', 'pretention', 'reconceptualize', 'cletis', 'kxch', 'montains', 'crowdpleaser', "'sy'", 'odete', 'edgaar', 'discoveres', 'techweb', "'it", 'nohe', "'stolen", 'mandalorians', 'talkiness', 'shakesperean', 'falsly', "'special", "effects'", "'searching", 'relatability', "spears'", 'saindo', 'dinheiro', 'piada', 'desgruda', 'gastou', 'encerra', 'meitlemeihr', "'smersh'", 'mutiple', 'proje', 'ningu', 'rmino', 'lhe', 'covardia', 'preocupe', 'necessidade', 'simbolizando', 'sciltre', 'verveen', 'captivatingly', "maenads'", "'stages'", "'extreme'", "chill'", "'deliverance'", 'unsurface', 'bauford', "'how", 'overmanipulative', 'unconned', "girls'", 'garmento', 'emotively', 'reeboir', "'gentle'", 'numerobis', "'my", "bickle'", 'pendance', 'bengtzon', 'maenwhile', 'shahirah', 'palnet', 'dreamplace', 'scyther', 'talisac', 'lucidique', 'm

In [162]:
len(data_vocab.keys())

18309

In [166]:
def create_embedding_matrix(word_vecs, data_vocab, min_df=2, D=300):
    """Creates embedding matrix from word vectors. """
    data_vocab = delete_rare_words(word_vecs, data_vocab, min_df)
    V = len(data_vocab.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, D), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(D, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25,0.25,D)
    vocab2index["UNK"] = 1
    i = 2
    for word in data_vocab:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25,D)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [167]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, data_vocab)

[]


In [168]:
len(pretrained_weight) # note that index 0 is for padding

18311

In [169]:
D = 300
V = len(pretrained_weight)
emb = nn.Embedding(V, D)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1108, -0.0502,  0.1493,  ..., -0.2310, -0.1393, -0.0501],
        [-0.0977, -0.5308, -0.2896,  ...,  0.4010, -0.1188,  0.1739],
        ...,
        [-0.3170, -0.4958,  0.3020,  ..., -0.1990,  0.0607, -0.1257],
        [-0.1369, -0.0570, -0.1921,  ..., -0.2036, -0.4955, -0.2766],
        [ 0.2028, -0.3397, -0.1055,  ...,  0.5841, -0.4893,  0.0245]])

Question: How many parameters do we have in this embedding matrix?

## Encoding training and validation sets

We will be using 1D Convolutional neural networks as our model. CNNs assume a fixed input size so we need to assume a fixed size and truncate or pad the sentences as needed. Let's find a good value to set our sequence length to.

In [170]:
x_len = np.array([len(x.split()) for x in X_train])

In [171]:
np.percentile(x_len, 95) # let set the max sequence len to N=40

42.0

In [172]:
X_train[0]

'will god let her fall or give her a new path \\?'

In [173]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("will", vocab2index["UNK"])

9

In [174]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 9,  2,  3, 11,  8,  5, 10, 11,  4,  7, 12,  6])

In [175]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [176]:
encode_sentence(X_train[0])

array([ 9,  2,  3, 11,  8,  5, 10, 11,  4,  7, 12,  6,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [177]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [178]:
x_test = np.vstack([encode_sentence(x) for x in X_test])
x_test.shape

(2000, 40)

## Playing and debugging CNN layers

In [179]:
V = len(pretrained_weight)
D = 300
N = 40

In [180]:
emb = nn.Embedding(V, D)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1108, -0.0502,  0.1493,  ..., -0.2310, -0.1393, -0.0501],
        [-0.0977, -0.5308, -0.2896,  ...,  0.4010, -0.1188,  0.1739],
        ...,
        [-0.3170, -0.4958,  0.3020,  ..., -0.1990,  0.0607, -0.1257],
        [-0.1369, -0.0570, -0.1921,  ..., -0.2036, -0.4955, -0.2766],
        [ 0.2028, -0.3397, -0.1055,  ...,  0.5841, -0.4893,  0.0245]])

In [181]:
x = x_train[:2]
x.shape

(2, 40)

In [182]:
x = torch.LongTensor(x)
x

tensor([[  9,   2,   3,  11,   8,   5,  10,  11,   4,   7,  12,   6,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [ 21,  23,  24,  27,  36,  14,  13,  19,  32,  35,  33,  22,
          25,  31,  16,  13,  28,  15,  26,  30,  34,  17,  20,  29,
          18,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0]])

In [183]:
x1 = emb(x)
x1.shape

torch.Size([2, 40, 300])

In [184]:
x1 = x1.transpose(1,2)  # needs to convert x to (batch, embedding_dim, sentence_len)
x1.size()

torch.Size([2, 300, 40])

In [185]:
conv_3 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=3)

In [186]:
x3 = conv_3(x1)

In [187]:
x3.size()

torch.Size([2, 100, 38])

In [188]:
conv_4 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=4)
conv_5 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=5)

In [189]:
x4 = conv_4(x1)
x5 = conv_5(x1)
print(x4.size(), x5.size())

torch.Size([2, 100, 37]) torch.Size([2, 100, 36])


Note that the convolution all apply to the same `x1`. How do we combine now the results of the convolutions? 

In [190]:
# 100 3-gram detectors
x3 = nn.ReLU()(x3)
x3 = nn.MaxPool1d(kernel_size = 38)(x3)
x3.size()

torch.Size([2, 100, 1])

In [191]:
# 100 4-gram detectors
x4 = nn.ReLU()(x4)
x4 = nn.MaxPool1d(kernel_size = 37)(x4)
x4.size()

torch.Size([2, 100, 1])

In [192]:
# 100 5-gram detectors
x5 = nn.ReLU()(x5)
x5 = nn.MaxPool1d(kernel_size = 36)(x5)
x5.size()

torch.Size([2, 100, 1])

In [193]:
# concatenate x3, x4, x5
out = torch.cat([x3, x4, x5], 2)
out.size()

torch.Size([2, 100, 3])

In [194]:
out = out.view(out.size(0), -1)
out.size()

torch.Size([2, 300])

After this we have a fully connected network. Let's write a network that implements this.

## 1D CNN model for sentence classification

Notation:
* V -- vocabulary size
* D -- embedding size
* N -- MAX Sentence length

In [195]:
class SentenceCNN(nn.Module):
    
    def __init__(self, V, D, glove_weights):
        super(SentenceCNN, self).__init__()
        self.glove_weights = glove_weights
        self.embedding = nn.Embedding(V, D, padding_idx=0)
        self.embedding.weight.data.copy_(torch.from_numpy(self.glove_weights))
        self.embedding.weight.requires_grad = False ## freeze embeddings

        self.conv_3 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=D, out_channels=100, kernel_size=5)
        
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(300, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1,2)
        x3 = F.relu(self.conv_3(x))
        x4 = F.relu(self.conv_4(x))
        x5 = F.relu(self.conv_5(x))
        x3 = nn.MaxPool1d(kernel_size = 38)(x3)
        x4 = nn.MaxPool1d(kernel_size = 37)(x4)
        x5 = nn.MaxPool1d(kernel_size = 36)(x5)
        out = torch.cat([x3, x4, x5], 2)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        return self.fc(out)   

In [196]:
V = len(pretrained_weight)
D = 300
N = 40
model = SentenceCNN(V, D, glove_weights=pretrained_weight)

In [197]:
# testing the model
x = x_train[:10]
print(x.shape)
x = torch.LongTensor(x)

(10, 40)


In [198]:
y_hat = model(x)
y_hat.size()

torch.Size([10, 1])

## Training

Note that I am not bodering with mini-batches since our dataset is small.

In [199]:
#def train(model, x_train, y_train):
model = SentenceCNN(V, D, glove_weights=pretrained_weight) #.cuda()

In [200]:
# accuracy of a random model should be around 0.5
test_metrics(model)

test loss 0.696 and accuracy 0.491


In [201]:
# this filters parameters with p.requires_grad=True
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [202]:
def train_epocs(model, epochs=10, lr=0.01):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    model.train()
    for i in range(epochs):
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item())
    test_metrics(model)

In [203]:
def test_metrics(m):
    model.eval()
    x = torch.LongTensor(x_test) #.cuda()
    y = torch.Tensor(y_test).unsqueeze(1) #).cuda()
    y_hat = m(x)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    print("test loss %.3f and accuracy %.3f" % (loss.item(), accuracy.item()))

In [204]:
train_epocs(model, epochs=10, lr=0.005)

0.6991532444953918
0.6992024779319763
0.602451503276825
0.46649008989334106
0.33072641491889954
0.36808934807777405
0.3891832232475281
0.3483543395996094
0.3032780587673187
0.2869540750980377
test loss 0.338 and accuracy 0.856


In [205]:
train_epocs(model, epochs=10, lr=0.001)

0.3003513514995575
0.27605175971984863
0.2787013053894043
0.27200281620025635
0.261476069688797
0.25204193592071533
0.24486103653907776
0.24786874651908875
0.2393726259469986
0.23336873948574066
test loss 0.272 and accuracy 0.887


In [206]:
train_epocs(model, epochs=10, lr=0.001)

0.22530794143676758
0.2191014289855957
0.21357125043869019
0.20763124525547028
0.20216995477676392
0.1929493546485901
0.18928399682044983
0.1845645010471344
0.1774616539478302
0.1737787425518036
test loss 0.243 and accuracy 0.901


In [207]:
# how to figure out the parameters
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 300, 4]), torch.Size([100]), torch.Size([100, 300, 5]), torch.Size([100]), torch.Size([1, 300]), torch.Size([1])]


### Unfreezing the embeddings

In [208]:
# unfreezing the embeddings
model.embedding.weight.requires_grad = True

In [209]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([18311, 300]), torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 300, 4]), torch.Size([100]), torch.Size([100, 300, 5]), torch.Size([100]), torch.Size([1, 300]), torch.Size([1])]


In [210]:
train_epocs(model, epochs=10, lr=0.001)

0.17019180953502655
0.15832334756851196
0.15441004931926727
0.1428917497396469
0.13642971217632294
0.12547792494297028
0.11840137094259262
0.11249307543039322
0.10614679753780365
0.09706176072359085
test loss 0.226 and accuracy 0.911


In [211]:
train_epocs(model, epochs=10, lr=0.0001)

0.09290824830532074
0.09156185388565063
0.09017505496740341
0.08888236433267593
0.08978015184402466
0.08727621287107468
0.08780866116285324
0.08726095408201218
0.08822084963321686
0.0855197012424469
test loss 0.222 and accuracy 0.911


## Lab 

* Improve tokenization
* Use fasttext instead of globe model. (https://fasttext.cc/docs/en/english-vectors.html)

   `! pip install git+https://github.com/facebookresearch/fastText.git`
* Extend this code to do cross-validation. Look at https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py for an example on how to do it.

## References

The CNN is adapted from here https://github.com/junwang4/CNN-sentence-classification-pytorch-2017/blob/master/cnn_pytorch.py.
Code for the original paper can be found here https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py.